In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import linear_model

import tensorflow as tf

%matplotlib inline

/Users/abulbasar/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df_train = pd.read_csv("/data/MNIST/mnist_train.csv", header=None)
df_test = pd.read_csv("/data/MNIST/mnist_test.csv", header=None)

y_train = df_train.iloc[:, 0].values
y_test = df_test.iloc[:, 0].values
X_train = df_train.iloc[:, 1:].values/255
X_test = df_test.iloc[:, 1:].values/255


In [3]:
tf.__version__

'1.6.0'

In [4]:
tf.reset_default_graph()
tf.set_random_seed(1)

n_x = X_train.shape[1]
n_y = len(np.unique(y_train))
n_h = 100

learning_rate = 0.001

X = tf.placeholder(dtype=tf.float32, shape=(None, n_x))
y = tf.placeholder(dtype=tf.int32, shape=(None,))
print("X", X)
print("y", y)

y_oh = tf.one_hot(y, depth=n_y)
print("y_oh", y_oh)

def dense_layer(input, size, activation = tf.nn.relu):
    input_size = int(input.get_shape()[1])
    W = tf.Variable(tf.truncated_normal(shape=(input_size, size), stddev=0.1))
    b = tf.Variable(tf.zeros(shape=(size,)))
    Z = tf.matmul(input, W) + b
    
    if callable(activation):
        Z = activation(Z)
    return Z


A1 = dense_layer(X, n_h)
print("A1", A1)

Z_out = dense_layer(A1, n_y, activation = None)
print("Z_out", Z_out)


y_pred = tf.argmax(Z_out, axis=1, output_type=tf.int32)
print("y_pred", y_pred)

match = tf.equal(y_pred, y)
print("match", match)

accuracy = tf.reduce_mean(tf.cast(match, tf.float32))
print("accuracy", accuracy)

loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_oh, logits=Z_out)
print("loss", loss)

cost = tf.reduce_mean(loss)

op = tf.train.AdamOptimizer(learning_rate).minimize(cost)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for i in range(100):
        _, cost_ = sess.run([op, cost], feed_dict = {X: X_train, y: y_train})
        
        print(cost_)    
    
    train_accuracy = sess.run(accuracy, feed_dict = {X: X_train, y: y_train})
    print("Test accuracy: ", train_accuracy)
    
    test_accuracy = sess.run(accuracy, feed_dict = {X: X_test, y: y_test})
    print("Test accuracy: ", test_accuracy)

X Tensor("Placeholder:0", shape=(?, 784), dtype=float32)
y Tensor("Placeholder_1:0", shape=(?,), dtype=int32)
y_oh Tensor("one_hot:0", shape=(?, 10), dtype=float32)
A1 Tensor("Relu:0", shape=(?, 100), dtype=float32)
Z_out Tensor("add_1:0", shape=(?, 10), dtype=float32)
y_pred Tensor("ArgMax:0", shape=(?,), dtype=int32)
match Tensor("Equal:0", shape=(?,), dtype=bool)
accuracy Tensor("Mean:0", shape=(), dtype=float32)
loss Tensor("softmax_cross_entropy_with_logits/Reshape_2:0", shape=(?,), dtype=float32)
2.3459167
2.2371323
2.1377778
2.0465329
1.9619112
1.8822734
1.8061674
1.732397
1.6602042
1.5892336
1.5194873
1.4512695
1.3849896
1.3210562
1.2598606
1.2016162
1.1464095
1.0942807
1.0452243
0.9992177
0.9562212
0.91613793
0.8788108
0.84405315
0.8116923
0.7815413
0.7534504
0.7272983
0.70296615
0.6803379
0.6592984
0.63972616
0.62150586
0.6045241
0.5886688
0.5738583
0.560011
0.5470528
0.53490615
0.5235032
0.51278275
0.5026907
0.4931858
0.48422965
0.47578597
0.46780977
0.46026063
0.45310378
0.

In [6]:
import keras

Using TensorFlow backend.


In [9]:
eye = np.eye(n_y)

In [10]:
def build_model():
    model = keras.Sequential()
    model.add(keras.layers.Dense(units=400, activation="relu", input_shape= (784,)))
    model.add(keras.layers.Dense(units=100, activation="relu"))
    model.add(keras.layers.Dense(units=n_y, activation="softmax"))
    model.compile(loss="categorical_crossentropy", metrics = ["accuracy"], optimizer = "adam")
    return model

model = build_model()
model.fit(X_train, eye[y_train], validation_data=(X_test, eye[y_test]), batch_size=32)
    

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 16s 268us/step - loss: 0.1985 - acc: 0.9412 - val_loss: 0.1027 - val_acc: 0.9670


In [11]:
def build_model():
    model = keras.Sequential()
    model.add(keras.layers.Dense(units=400, activation="tanh", input_shape= (784,)))
    model.add(keras.layers.Dense(units=100, activation="tanh"))
    model.add(keras.layers.Dense(units=n_y, activation="softmax"))
    model.compile(loss="categorical_crossentropy", metrics = ["accuracy"], optimizer = "adam")
    return model

model = build_model()
model.fit(X_train, eye[y_train], validation_data=(X_test, eye[y_test]), batch_size=32)
    

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 16s 259us/step - loss: 0.2364 - acc: 0.9291 - val_loss: 0.1327 - val_acc: 0.9597


In [17]:
def build_model():
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(filters=32, kernel_size=(5, 5), activation="relu", input_shape= (28,28,1)))
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu"))
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(units=400, activation="relu", ))
    model.add(keras.layers.Dense(units=100, activation="relu"))
    model.add(keras.layers.Dense(units=n_y, activation="softmax"))
    model.compile(loss="categorical_crossentropy", metrics = ["accuracy"], optimizer = "adam")
    return model

model = build_model()
model.summary()
model.fit(X_train.reshape((-1, 28, 28, 1))
          , eye[y_train]
          , epochs=10
          , validation_data=(X_test.reshape((-1, 28, 28, 1)), eye[y_test])
          , batch_size=32)
    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 10, 10, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_22 (Dense)             (None, 400)               640400    
_________________________________________________________________
dense_23 (Dense)             (None, 100)               40100     
__________